In [1]:
import glob
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import calendar
from collections import Counter

## Functions

In [3]:
def convert_date(date):
    # Remove special characters
    date = re.sub('[^a-z0-9]', " ", date)
    try:
        # ASSUMING dd MMM
        # Extract day
        day_start = re.search('^\d+', date).start()
        day_end = re.search('^\d+', date).end()
        day = date[day_start:day_end]
        if len(day) == 1:
            day = "0"+day
        # Extract month
        date_wo_day = date[day_end:].strip()
        month_start = re.search('^\w+', date_wo_day).start()
        month_end = re.search('^\w+', date_wo_day).end()
        month = date_wo_day[month_start:month_end]
        if "jan" in str(month):
            month = "01"
        if "feb" in str(month):
            month = "02"
        if "mar" in str(month):
            month = "03"
        if "apr" in str(month):
            month = "04"
        if "may" in str(month):
            month = "05"
        if "jun" in str(month):
            month = "06"
        if "jul" in str(month):
            month = "07"
        if "aug" in str(month):
            month = "08"
        if "sep" in str(month):
            month = "09"
        if "oct" in str(month):
            month = "10"
        if "nov" in str(month):
            month = "11"
        if "dec" in str(month):
            month = "12"

        full_date = day+"."+month+".2019"
        if len(full_date) == 10:
            return datetime.strptime(full_date, '%d.%m.%Y').date()
        else:
            return "nil"
    except:
        return "nil"

In [4]:
def convert_time(time, post_time):
    return_time = "nil"
    time_original = str(re.sub(r'[.:-]', '', time)).strip()
    
    # ASSUME NUM PM
    if bool(re.search(r'^\d+ ?pm', time_original)):
        time = time_original.replace("pm", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "13:00"
            if time == 2:
                return_time = "14:00"
            if time == 3:
                return_time = "15:00"
            if time == 4:
                return_time = "16:00"
            if time == 5:
                return_time = "17:00"
            if time == 6:
                return_time = "18:00"
            if time == 7:
                return_time = "19:00"
            if time == 8:
                return_time = "20:00"
            if time == 9:
                return_time = "21:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "22:00"
            if time == 11:
                return_time = "23:00"
            if time == 12:
                return_time = "12:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "13:"+time[1:]
            if hour == 2:
                return_time = "14:"+time[1:]
            if hour == 3:
                return_time = "15:"+time[1:]
            if hour == 4:
                return_time = "16:"+time[1:]
            if hour == 5:
                return_time = "17:"+time[1:]
            if hour == 6:
                return_time = "18:"+time[1:]
            if hour == 7:
                return_time = "19:"+time[1:]
            if hour == 8:
                return_time = "20:"+time[1:]
            if hour == 9:
                return_time = "21:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "22:"+time[2:]
            if hour == 11:
                return_time = "23:"+time[2:]
            if hour == 12:
                return_time = "12:"+time[2:]
    
    # ASSUME NUM AM
    if bool(re.search(r'^\d+ ?am', time_original)):
        time = time_original.replace("am", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "01:00"
            if time == 2:
                return_time = "02:00"
            if time == 3:
                return_time = "03:00"
            if time == 4:
                return_time = "04:00"
            if time == 5:
                return_time = "05:00"
            if time == 6:
                return_time = "06:00"
            if time == 7:
                return_time = "07:00"
            if time == 8:
                return_time = "08:00"
            if time == 9:
                return_time = "09:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "10:00"
            if time == 11:
                return_time = "11:00"
            if time == 12:
                return_time = "00:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "01:"+time[1:]
            if hour == 2:
                return_time = "02:"+time[1:]
            if hour == 3:
                return_time = "03:"+time[1:]
            if hour == 4:
                return_time = "04:"+time[1:]
            if hour == 5:
                return_time = "05:"+time[1:]
            if hour == 6:
                return_time = "06:"+time[1:]
            if hour == 7:
                return_time = "07:"+time[1:]
            if hour == 8:
                return_time = "08:"+time[1:]
            if hour == 9:
                return_time = "09:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "10:"+time[2:]
            if hour == 11:
                return_time = "11:"+time[2:]
            if hour == 12:
                return_time = "00:"+time[2:]
    
    # ASSUME NO PM OR AM
    if bool(re.search(r'^\d+$', time_original)):
        time = str(time_original).strip()
        # If single digit
        if len(time) == 1:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 1:
                    return_time = "13:00"
                if time == 2:
                    return_time = "14:00"
                if time == 3:
                    return_time = "15:00"
                if time == 4:
                    return_time = "16:00"
                if time == 5:
                    return_time = "17:00"
                if time == 6:
                    return_time = "18:00"
                if time == 7:
                    return_time = "19:00"
                if time == 8:
                    return_time = "20:00"
                if time == 9:
                    return_time = "21:00"
            else:
                if time == 1:
                    return_time = "01:00"
                if time == 2:
                    return_time = "02:00"
                if time == 3:
                    return_time = "03:00"
                if time == 4:
                    return_time = "04:00"
                if time == 5:
                    return_time = "05:00"
                if time == 6:
                    return_time = "06:00"
                if time == 7:
                    return_time = "07:00"
                if time == 8:
                    return_time = "08:00"
                if time == 9:
                    return_time = "09:00"
        # If double digits
        if len(str(time)) == 2:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 10:
                    return_time = "22:00"
                if time == 11:
                    return_time = "23:00"
                if time == 12:
                    return_time = "12:00"
            else:
                if time == 10:
                    return_time = "10:00"
                if time == 11:
                    return_time = "11:00"
                if time == 12:
                    return_time = "00:00"    
        # If three digtis
        if len(str(time)) == 3:
            hour = int(str(time)[0])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour == 1:
                    return_time = "13:"+time[1:]
                if hour == 2:
                    return_time = "14:"+time[1:]
                if hour == 3:
                    return_time = "15:"+time[1:]
                if hour == 4:
                    return_time = "16:"+time[1:]
                if hour == 5:
                    return_time = "17:"+time[1:]
                if hour == 6:
                    return_time = "18:"+time[1:]
                if hour == 7:
                    return_time = "19:"+time[1:]
                if hour == 8:
                    return_time = "20:"+time[1:]
                if hour == 9:
                    return_time = "21:"+time[1:]
            else:
                if hour == 1:
                    return_time = "01:"+time[1:]
                if hour == 2:
                    return_time = "02:"+time[1:]
                if hour == 3:
                    return_time = "03:"+time[1:]
                if hour == 4:
                    return_time = "04:"+time[1:]
                if hour == 5:
                    return_time = "05:"+time[1:]
                if hour == 6:
                    return_time = "06:"+time[1:]
                if hour == 7:
                    return_time = "07:"+time[1:]
                if hour == 8:
                    return_time = "08:"+time[1:]
                if hour == 9:
                    return_time = "09:"+time[1:]
        # If four digtis
        if len(str(time)) == 4:
            hour = int(str(time)[0:2])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour >= 12:
                    return_time = str(hour)+":"+str(time)[2:]
                if hour == 10:
                    return_time = "22:"+time[2:]
                if hour == 11:
                    return_time = "23:"+time[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
            else:
                if hour <= 12:
                    if len(str(hour)) == 1:
                        return_time = "0"+str(hour)+":"+str(time)[2:]
                    else:
                        return_time = str(hour)+":"+str(time)[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
                if hour == 10:
                    return_time = "10:"+time[2:]
                if hour == 11:
                    return_time = "11:"+time[2:]
                if hour == 12:
                    return_time = "00:"+time[2:]
        
    return return_time

## Process Data

### Read Files

In [5]:
html_files_old = glob.glob("./telegram_chat_export_beginning_19sep/*.html")
print("Number of files: ", len(html_files_old))
print(html_files_old[:10])
print()

html_files_new = glob.glob("./telegram_chat_export_13oct_6dec/*.html")
print("Number of files: ", len(html_files_new))
print(html_files_new[:10])

Number of files:  257
['./telegram_chat_export_beginning_19sep/messages69.html', './telegram_chat_export_beginning_19sep/messages198.html', './telegram_chat_export_beginning_19sep/messages109.html', './telegram_chat_export_beginning_19sep/messages82.html', './telegram_chat_export_beginning_19sep/messages4.html', './telegram_chat_export_beginning_19sep/messages113.html', './telegram_chat_export_beginning_19sep/messages175.html', './telegram_chat_export_beginning_19sep/messages34.html', './telegram_chat_export_beginning_19sep/messages93.html', './telegram_chat_export_beginning_19sep/messages14.html']

Number of files:  148
['./telegram_chat_export_13oct_6dec/messages69.html', './telegram_chat_export_13oct_6dec/messages109.html', './telegram_chat_export_13oct_6dec/messages82.html', './telegram_chat_export_13oct_6dec/messages4.html', './telegram_chat_export_13oct_6dec/messages113.html', './telegram_chat_export_13oct_6dec/messages34.html', './telegram_chat_export_13oct_6dec/messages93.html'

In [6]:
html_files = [*html_files_old, *html_files_new]
print("Total html files: ", len(html_files))

Total html files:  405


### Extract Data from Files

In [10]:
messages_list = []
total_messages = 0

for file in tqdm(html_files):    
    f = open(file, "r", encoding='utf-8')
    chat = f.read()
    soup = BeautifulSoup(chat)
    messages = soup.findAll("div", {"class": "message default clearfix"})
    total_messages += len(messages)

    for message in messages:
        # Get user
        user = message.find("div", {"class": "from_name"}).text.replace("\n", "").strip()
        #print(user)
        # Get post date and time
        post_datetime = message.find("div", {"class": "pull_right date details"}).get("title")
        post_date = post_datetime.split(" ")[0]
        post_time = post_datetime.split(" ")[1]
        #print(post_date)
        #print(post_time)
        # GET MESSAGE
        raw_message = message.find("div", {"class": "text"})
        # Extract text from message
        raw_message = str(raw_message).replace("</div>", "").replace('<div class="text">', "").replace("<strong>", "").replace("</strong>", "").replace("<u>", "").replace("</u>", "").strip().lower()
        # Tokenize text
        raw_message_tokens = re.sub(r'(<br/> ?){2,}', '<br/>', raw_message).split("<br/>")

        # Variable flag for actual ride request
        is_request = False
        # Get user type
        if(bool(re.search(r'hitchers? looking', raw_message_tokens[0]))):
            user_type = "hitcher"
            is_request = True
        elif(bool(re.search(r'driver? looking', raw_message_tokens[0]))):
            user_type = "driver"
            is_request = True
        else:
            user_type = "nil"

        # PROCESS MESSAGE CONTENT IF IT IS ACTUALLY A REQUEST
        if(is_request):
            # Blank place holder
            pick_up = "nil"
            drop_off = "nil"
            request_date = "nil"
            request_time = "nil"
            number_of_pax = "nil"
            request_date_normalised = "nil"
            request_time_normalised = "nil"
            day = "nil"
            # ------------------

            # Extract pick up location assuming it is always the second token
            try:
                pick_up_tokens = str(raw_message_tokens[1]).split(":")
            except:
                pass
            try:
                if "pick" in pick_up_tokens[0]:
                    pick_up = pick_up_tokens[1].strip()
            except:
                pick_up_tokens = str(raw_message_tokens[1]).split(" ")
                if "pick" in pick_up_tokens[0]:
                    pick_up = " ".join(pick_up_tokens[1:])
                else:
                    pass

            # Extract drop off location assuming it is always the third token
            try:
                drop_off_tokens = str(raw_message_tokens[2]).split(":")
            except:
                pass
            try:
                if "drop" in drop_off_tokens[0]:
                    drop_off = drop_off_tokens[1].strip()
            except:
                drop_off_tokens = str(raw_message_tokens[2]).split(" ")
                if "drop" in drop_off_tokens[0]:
                    drop_off = " ".join(drop_off_tokens[1:])
                else:
                    pass

            # Extract date assuming it is always the fourth token
            try:
                date_tokens = str(raw_message_tokens[3]).split(":")
            except:
                pass
            try:
                if "date" in date_tokens[0]:
                    request_date = date_tokens[1].strip()
            except:
                date_tokens = str(raw_message_tokens[3]).split(" ")
                if "date" in date_tokens[0]:
                    request_date = " ".join(date_tokens[1:])
                else:
                    pass
            if "today" in request_date or "tdy" in request_date:
                request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
            elif "tmr" in request_date or "tomorrow" in request_date:
                today_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                tomorrow_date = today_date + timedelta(days=1)
                request_date = tomorrow_date
            elif request_date == "nil":
                pass
            else:
                request_date = convert_date(request_date)
            
            # Extract time assuming it is always the fifth token
            try:
                time_raw = str(raw_message_tokens[4])
            except:
                pass
            try:
                time = time_raw.replace("time:", "").replace("time :", "").strip()
                time = re.sub(r'[^a-z0-9.-:]', '', time).strip()
                if "now" in time:
                    request_time = post_time[:5]
                else:
                    request_time = convert_time(str(time), post_time)
            except:
                pass

            # Extract number of pax assuming it is always the sixth token
            try:
                if "pax" in raw_message_tokens[5]:
                    raw_pax = str(raw_message_tokens[5]).replace("pax:", "").replace("pax", "").replace("pax :", "").replace("pax : ", "").strip()
                    raw_pax = raw_pax.replace(":", "").strip()
                    raw_pax = re.sub(r'\(.*\)', '', raw_pax)
                    raw_pax = re.sub(r'[^a-z0-9- ]', '', raw_pax)
                    raw_pax = raw_pax.strip()
                    if len(raw_pax) == 1:
                        number_of_pax = raw_pax
                    elif len(raw_pax) == 3:
                        number_of_pax = raw_pax
                    else:
                        if bool(re.search(r'^\d+ - \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace(" ", "")
                        if bool(re.search(r'^\d+ to \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace("to", "-")
                            number_of_pax = number_of_pax.replace(" ", "")
            except:
                pass
            
            
            #--------------------NORMALISE DATA---------------------
            
            # Normalise date and time
            try:
                date_time = datetime.strptime(str(request_date)+" "+request_time, '%Y-%m-%d %H:%M')
                approx = round(date_time.minute/30.0) * 30
                date_time = date_time.replace(minute=0)
                date_time += timedelta(seconds=approx*60)
                request_date_normalised = date_time.date()
                request_time_normalised = date_time.strftime('%H:%M')
            except:
                pass
            
            # Add day to data eg. dd-mm-yyyy = wed
            try:
                my_date = datetime.strptime(str(request_date_normalised), '%Y-%m-%d')
                day = calendar.day_name[my_date.weekday()].lower()
            except:
                pass
            
            # Store structured message into dictionary
            message_data = {
                'user' : user,
                'post_date' : post_date,
                'post_time' : post_time,
                'raw_message' : raw_message,
                'user_type' : user_type,
                'pick_up' : pick_up,
                'drop_off' : drop_off,
                'request_date' : request_date,
                'request_date_normalised' : request_date_normalised,
                'request_time' : request_time,
                'request_time_normalised' : request_time_normalised,
                'day' : day,
                'number_of_pax' : number_of_pax
            }
            messages_list.append(message_data)

100%|██████████| 405/405 [04:23<00:00,  1.54it/s]


### Stats

In [11]:
print("Total number of messages: \t\t", total_messages)
print("Total number of actual messages: \t", len(messages_list))
print()
print("Percentage of actual messages: \t", round((len(messages_list)/total_messages)*100, 2), "%")

Total number of messages: 		 365298
Total number of actual messages: 	 349801

Percentage of actual messages: 	 95.76 %


### Convert Data to DataFrame

In [12]:
data = pd.DataFrame(messages_list)
data.head()
data.to_csv("./data/sg_hitch_data_raw.csv")

### Filter out Corrupted Data

In [2]:
data = pd.read_csv("./data/sg_hitch_data_raw.csv", index_col=0)

# Filter out corrupted request time
clean_data_rt = data[data.request_time_normalised != "nil"]
print("Removed", len(data)-len(clean_data_rt), "corrupted request time")
# Filter out corrupted request date
clean_data_rd = clean_data_rt[clean_data_rt.request_date_normalised != "nil"]
print("Removed", len(clean_data_rt)-len(clean_data_rd), "corrupted request date")
# Filter out corrupted pick up
clean_data_pu = clean_data_rd[clean_data_rd.pick_up != "nil"]
print("Removed", len(clean_data_rd)-len(clean_data_pu), "corrupted pick up")
# Filter out corrupted drop off
clean_data_do = clean_data_pu[clean_data_pu.drop_off != "nil"]
print("Removed", len(clean_data_pu)-len(clean_data_do), "corrupted drop off")

Removed 52874 corrupted request time
Removed 0 corrupted request date
Removed 81 corrupted pick up
Removed 1763 corrupted drop off


In [3]:
clean_data = clean_data_do
print("Number of requests left: ", len(clean_data))
print()
print("Percentage of non-corrupted data: ", round(len(clean_data)/len(data)*100, 2), "%")

Number of requests left:  295083

Percentage of non-corrupted data:  84.36 %


In [4]:
clean_data.tail()

,user,post_date,post_time,raw_message,user_type,pick_up,drop_off,request_date,request_date_normalised,request_time,request_time_normalised,day,number_of_pax
349796,jerms 😚,23.11.2019,18:08:54,👋🏻 hitcher looking for driver<br/>pick up: six...,hitcher,sixth ave,hougang,2019-11-23,2019-11-23,18:08,18:00,saturday,1
349797,4min,23.11.2019,18:09:00,👋 hitcher looking for driver<br/>pick up: sk<b...,hitcher,sk,forum,2019-11-23,2019-11-23,18:09,18:00,saturday,1
349798,johnwi,23.11.2019,18:11:11,👋 hitcher looking for driver<br/>pick up: sing...,hitcher,singapore sports school,tampines mall,2019-11-23,2019-11-23,18:11,18:00,saturday,4
349799,Branden,23.11.2019,18:12:44,👋🏻hitcher looking for driver<br/>pick up : 791...,hitcher,791450,pomo (dhoby ghaut),2019-11-23,2019-11-23,18:12,18:00,saturday,1
349800,Yx,23.11.2019,18:13:57,🚗driver looking for hitcher<br/>pick up: wood...,driver,woodlands,any where,2019-11-23,2019-11-23,18:13,18:00,saturday,1-4


### Normalise Data

In [9]:
# Normalise Location
pap = pd.read_csv("./data/planning_area_population.csv")
planning_areas = pap['name'].tolist()
len(planning_areas)

55

In [10]:
pick_up = clean_data['pick_up'].tolist()
drop_off = clean_data['drop_off'].to_list()

In [16]:
def clean_location(loc):
    clean = re.sub(r'\bamk\b', 'ang mo kio', loc)
    clean = re.sub(r'\bcck\b', 'choa chu kang', clean)
    clean = re.sub(r'\btpy\b', 'toa payoh', clean)
    clean = re.sub(r'\byck\b', 'yio chu kang', clean)
    lean = re.sub(r'^vivo$', 'vivo city', clean)
    clean = re.sub(r'\btamp\b', 'tampines', clean)
    clean = re.sub(r'\bseng kang\b', 'sengkang', clean)
    clean = re.sub(r'\bcq\b', 'clarke quay', clean)
    return clean

In [19]:
pick_up_clean = []
drop_off_clean = []

for pu in pick_up:
    pick_up_clean.append(clean_location(str(pu)))
    
for do in drop_off:
    drop_off_clean.append(clean_location(str(do)))

In [25]:
print(Counter(pick_up_clean).most_common(20))
print(Counter(drop_off_clean).most_common(20))

[('woodlands', 6884), ('yishun', 6400), ('tampines', 6031), ('sengkang', 5862), ('punggol', 4610), ('hougang', 4535), ('ang mo kio', 4249), ('choa chu kang', 4232), ('pasir ris', 4198), ('jurong west', 3357), ('bedok', 3003), ('jurong', 2666), ('bukit batok', 2539), ('sembawang', 2525), ('bukit panjang', 2360), ('toa payoh', 1510), ('town', 1483), ('serangoon', 1386), ('clarke quay', 1325), ('clementi', 1315)]
[('anywhere', 7714), ('woodlands', 6513), ('yishun', 6148), ('tampines', 5786), ('sengkang', 5274), ('punggol', 4666), ('hougang', 4442), ('ang mo kio', 3908), ('pasir ris', 3904), ('choa chu kang', 3873), ('jurong west', 3328), ('bedok', 2922), ('jurong', 2903), ('bukit batok', 2350), ('bukit panjang', 2330), ('sembawang', 2323), ('town', 2076), ('clementi', 1517), ('toa payoh', 1450), ('ngee ann poly', 1303)]


In [ ]:
pick_up_normalised = []
drop_off_normalised = []

for i in tqdm(range(len(pick_up))):
    for area in planning_areas:
        try:
            if area in pick_up[i]:
                #print(area, " -> ", pick_up[i])
                pick_up_normalised.append(area)
        except:
            pick_up_normalised.append(pick_up[i])
        try:
            if area in drop_off[i]:
                drop_off_normalised.append(area)
        except:
            drop_off_normalised.append(drop_off[i])

  0%|                                                                                       | 0/295083 [00:00<?, ?it/s]

novena  ->  novena square 2,main entrance near jollibee
bukit panjang  ->  bukit panjang
yishun  ->  yishun blk 212
yishun  ->  yishun blk 212
bukit merah  ->  bukit merah lane 1
woodlands  ->  woodlands
yishun  ->  yishun blk 212
hougang  ->  hougang
yishun  ->  yishun blk 212
woodlands  ->  woodlands
hougang  ->  the midtown (hougang)
bedok  ->  bedok
yishun  ->  yishun blk 212
hougang  ->  the midtown (hougang)
bukit panjang  ->  bukit panjang
pasir ris  ->  pasir ris
seletar  ->  seletar
yishun  ->  yishun blk 212
yishun  ->  yishun blk 212
hougang  ->  the midtown (hougang)
hougang  ->  hougang
yishun  ->  yishun blk 212
bukit merah  ->  bukit merah lane 1
yishun  ->  yishun blk 212
clementi  ->  clementi
yishun  ->  yishun blk 212
punggol  ->  punggol walk
bukit batok  ->  bukit batok
bukit merah  ->  bukit merah lane 1


  0%|                                                                             | 50/295083 [00:00<10:01, 490.47it/s]

yishun  ->  yishun blk 212
sembawang  ->  woodlands / sembawang
woodlands  ->  woodlands / sembawang
bukit batok  ->  bukit batok
yishun  ->  yishun blk 212
bukit panjang  ->  bukit panjang
jurong east  ->  jurong east
bukit batok  ->  bukit batok
tampines  ->  tampines central
bishan  ->  9 clover road (bishan)
yishun  ->  yishun
yishun  ->  yishun
clementi  ->  clementi mall
woodlands  ->  737771 woodlands drive 16
yishun  ->  yishun blk 212
woodlands  ->  737771 woodlands drive 16
punggol  ->  punggol
yishun  ->  yishun
punggol  ->  punggol/seng kang
tampines  ->  tampines
bedok  ->  137 bedok north ave 3
yishun  ->  yishun
bukit merah  ->  bukit merah lane 1
pasir ris  ->  pasir ris
bedok  ->  137 bedok north ave 3
pasir ris  ->  pasir ris
changi  ->  jewel changi
pasir ris  ->  pasir ris


  0%|                                                                            | 132/295083 [00:00<08:49, 556.64it/s]

...

woodlands  ->  woodlands
hougang  ->  hougang
yishun  ->  yishun
yishun  ->  yishun
woodlands  ->  woodlands
choa chu kang  ->  choa chu kang street 62
novena  ->  novena
bukit batok  ->  bukit batok crescent
sembawang  ->  sembawang
bedok  ->  bedok reservoir road
bukit batok  ->  bukit batok crescent
choa chu kang  ->  choa chu kang  ave 4 blk 446
choa chu kang  ->  20 choa chu kang street 64
queenstown  ->  queenstown
yishun  ->  yishun
orchard  ->  forum galleria orchard
woodlands  ->  woodlands mrt
sengkang  ->  sengkang
orchard  ->  orchard
woodlands  ->  woodlands
jurong west  ->  jurong west central 1
woodlands  ->  woodlands
clementi  ->  clementi
paya lebar  ->  certis cisco paya lebar
sengkang  ->  sengkang
clementi  ->  clementi west
jurong west  ->  jurong west 648351
orchard  ->  hilton hotel orchard
tampines  ->  tampines ave 8 520878
jurong west  ->  jurong west
bukit batok  ->  bukit batok
tampines  ->  tampines ave 8 520878
jurong west  ->  jurong west
tampines  ->  t

 32%|███████████████████████▌                                                  | 93980/295083 [02:07<04:19, 775.72it/s]

jurong west  ->  jurong west
serangoon  ->  6 serangoon avenue 5
bedok  ->  bedok
yishun  ->  yishun 462a or near by
jurong west  ->  jurong west
tampines  ->  tampines mart
serangoon  ->  6 serangoon avenue 5
yishun  ->  yishun 462a or near by
bukit batok  ->  bukit batok
tampines  ->  tampines mart
toa payoh  ->  toa payoh
jurong east  ->  600318 (jurong east)
jurong east  ->  600318 (jurong east)
jurong west  ->  jurong west
pasir ris  ->  pasir ris/tampines
tampines  ->  pasir ris/tampines
jurong east  ->  600318 (jurong east)
bukit panjang  ->  bukit panjang
tampines  ->  tampines
jurong east  ->  600318 (jurong east)
jurong east  ->  600318 (jurong east)
hougang  ->  530336 (hougang)
pasir ris  ->  pasir ris/tampines
tampines  ->  pasir ris/tampines
bukit panjang  ->  bukit panjang
bukit panjang  ->  bukit panjang
serangoon  ->  6 serangoon avenue 5
ang mo kio  ->  ang mo kio
jurong east  ->  jurong east
punggol  ->  punggol
yishun  ->  yishun
jurong east  ->  jurong east
woodlan

 32%|███████████████████████▌                                                  | 94093/295083 [02:07<03:55, 852.49it/s]

yishun  ->  yishun chompang
hougang  ->  hougang
changi  ->  changi
jurong west  ->  jurong west
tampines  ->  tampines
bedok  ->  bedok
pasir ris  ->  pasir ris blk 515
jurong west  -> 

In [28]:
pick_up_normalised

['novena',
 'bukit panjang',
 'yishun',
 'yishun',
 'bukit merah',
 'woodlands',
 'yishun',
 'hougang',
 'yishun',
 'woodlands',
 'hougang',
 'bedok',
 'yishun',
 'hougang',
 'bukit panjang',
 'pasir ris',
 'seletar',
 'yishun',
 'yishun',
 'hougang',
 'hougang',
 'yishun',
 'bukit merah',
 'yishun',
 'clementi',
 'yishun',
 'punggol',
 'bukit batok',
 'bukit merah',
 'yishun',
 'sembawang',
 'woodlands',
 'bukit batok',
 'yishun',
 'bukit panjang',
 'jurong east',
 'bukit batok',
 'tampines',
 'bishan',
 'yishun',
 'yishun',
 'clementi',
 'woodlands',
 'yishun',
 'woodlands',
 'punggol',
 'yishun',
 'punggol',
 'tampines',
 'bedok',
 'yishun',
 'bukit merah',
 'pasir ris',
 'bedok',
 'pasir ris',
 'changi',
 'pasir ris',
 'yishun',
 'tampines',
 'changi',
 'changi',
 'bedok',
 'woodlands',
 'bukit panjang',
 'kallang',
 'bukit batok',
 'tampines',
 'yishun',
 'woodlands',
 'orchard',
 'orchard',
 'bishan',
 'bukit timah',
 'bishan',
 'changi',
 'yishun',
 'jurong west',
 'yishun',
 'y

In [16]:
clean_data.to_csv("./data/sg_hitch_data.csv")